In [2]:
import pandas as pd
import os
%xmode minimal

Exception reporting mode: Minimal


In [3]:
url = 'https://raw.githubusercontent.com/pandas-dev/pandas/master/doc/data/titanic.csv'
file = '../data/titanic_2_original.pickle' # Damit es local gespeichert wird

if os.path.exists(file):
    t = pd.read_pickle(file)
else:
    t = pd.read_csv(url)
    t.to_pickle(file)
t.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
416,417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",female,34.0,1,1,28220,32.5000,NaN,S
433,434,0,3,"Kallio, Mr. Nikolai Erland",male,17.0,0,0,STON/O 2. 3101274,7.1250,NaN,S
562,563,0,2,"Norman, Mr. Robert Douglas",male,28.0,0,0,218629,13.5000,NaN,S
99,100,0,2,"Kantor, Mr. Sinai",male,34.0,1,0,244367,26.0000,NaN,S
35,36,0,1,"Holverson, Mr. Alexander Oskar",male,42.0,1,0,113789,52.0000,NaN,S
65,66,1,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
455,456,1,3,"Jalsevac, Mr. Ivan",male,29.0,0,0,349240,7.8958,NaN,C
786,787,1,3,"Sjoblom, Miss. Anna Sofia",female,18.0,0,0,3101265,7.4958,NaN,S
507,508,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,NaN,0,0,111427,26.5500,NaN,S


## 4.1

In [4]:
def alter_bins(alter):
    if pd.isna(alter): return 'kein Alter'
    if alter < 11: return 'child'
    if alter <18 and alter >= 12: return 'teenager'
    if alter < 66 and alter > 17: return 'adult'
    if alter > 65: return 'adult'
    

In [5]:
t['alter_bin'] = t['Age'].apply(alter_bins)

In [6]:
t.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,alter_bin
573,574,1,3,"Kelly, Miss. Mary",female,NaN,0,0,14312,7.7500,NaN,Q,kein Alter
151,152,1,1,"Pears, Mrs. Thomas (Edith Wearne)",female,22.0,1,0,113776,66.6000,C2,S,adult
842,843,1,1,"Serepeca, Miss. Augusta",female,30.0,0,0,113798,31.0000,NaN,C,adult
509,510,1,3,"Lang, Mr. Fang",male,26.0,0,0,1601,56.4958,NaN,S,adult
843,844,0,3,"Lemberopolous, Mr. Peter L",male,34.5,0,0,2683,6.4375,NaN,C,adult


## 4.2

In [7]:
def alter_bins(alter,sex):
    if pd.isna(alter): return 'kein Alter'
    if alter < 11: return 'child'
    if alter <18 and alter >= 12: return 'teenager'
    if alter < 66 and alter > 17 and sex == 'female': return 'male'
    if alter < 66 and alter > 17 and sex == 'female': return 'male'
    if alter > 65: return 'adult'

In [8]:
def alter_bins(row):
    alter = row['Age']
    sex = row['Sex']

    if pd.isna(alter):
        return 'kein Alter'
    if alter < 11:
        return 'child'
    if 12 <= alter < 18:
        return 'teenager'
    if 18 <= alter < 66:
        return 'male' if sex == 'male' else 'female'  
    if alter >= 66:
        return 'adult'

# Anwenden der Funktion auf jede Zeile
t['alter_bin_sex'] = t.apply(alter_bins, axis=1)

In [9]:
t.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,alter_bin,alter_bin_sex
58,59,1,2,"West, Miss. Constance Mirium",female,5.0,1,2,C.A. 34651,27.7500,NaN,S,child,child
82,83,1,3,"McDermott, Miss. Brigdet Delia",female,NaN,0,0,330932,7.7875,NaN,Q,kein Alter,kein Alter
431,432,1,3,"Thorneycroft, Mrs. Percival (Florence Kate White)",female,NaN,1,0,376564,16.1000,NaN,S,kein Alter,kein Alter
563,564,0,3,"Simmons, Mr. John",male,NaN,0,0,SOTON/OQ 392082,8.0500,NaN,S,kein Alter,kein Alter
447,448,1,1,"Seward, Mr. Frederic Kimber",male,34.0,0,0,113794,26.5500,NaN,S,adult,male


## 4.3
<br>
Funktionen: <br>
In Quantile aufteilen : t['Quantil'], grenzen = pd.qcut(Spalte bsp t['Spalte'], q=Anzahl Quantile, labels=Liste mit labels, retbins=True)

In [10]:
quantile_labels = ['niedrig', 'normal', 'hoch', 'sehr hoch']
t['Quantil'], grenzen = pd.qcut(t['Fare'], q=4, labels=quantile_labels, retbins=True)

# Ergebnis anzeigen
print(grenzen)

[  0.       7.9104  14.4542  31.     512.3292]


c)

In [12]:
median = grenzen[2]
median

14.4542

In [15]:
labels = ['billig', 'normal', 'teuer', 'sehr_teuer']

t_grouped_class = t.groupby('Pclass')

In [22]:
t_grouped_class.sum()

/var/folders/vr/klm94mws7lsd0vj7h5gd2bqw0000gn/T/ipykernel_57258/136938376.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  t_grouped_class.sum()


,PassengerId,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,,
1,99705,136,7111.42,90,77,18177.4125
2,82056,87,5168.83,74,70,3801.8417
3,215625,119,8924.92,302,193,6714.6951


In [24]:
t_grouped_class.iloc[0]

AttributeError: 'DataFrameGroupBy' object has no attribute 'iloc'

In [27]:
# Definiere eine Funktion, die die Quantilschnitt (qcut) auf jede Gruppe anwendet
def quantile_cut(group):
    return pd.qcut(group, q=4, labels=labels, duplicates='drop')

# Gruppiere nach 'Pclass' und wende die Funktion auf die 'Fare'-Spalte an
t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(quantile_cut)

/var/folders/vr/klm94mws7lsd0vj7h5gd2bqw0000gn/T/ipykernel_57258/1720030728.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(quantile_cut)


In [34]:
# Gruppiere nach 'Pclass' und wende die Funktion auf die 'Fare'-Spalte an
t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(lambda s: pd.qcut(s, q=4, labels=labels))

/var/folders/vr/klm94mws7lsd0vj7h5gd2bqw0000gn/T/ipykernel_57258/178223049.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  t['Fare Bins'] = t.groupby('Pclass')['Fare'].apply(lambda s: pd.qcut(s, q=4, labels=labels))


## 4.4

In [38]:
def alone(c):
    if c['Parch'] > 0 or c['SibSp'] > 0: return 0
    if c['Parch'] == 0 or c['SibSp'] == 0: return 1

0      0
1      0
2      0
3      0
4      0
      ..
886    0
887    0
888    2
889    0
890    0
Name: Parch, Length: 891, dtype: int64

In [39]:
t['Alone'] = t.apply(alone, axis=1)